Projeto ETL - Livro Bingo Akatsuki (Santander Dev Week)
Este notebook demonstra um pipeline de Data Engineering seguindo o fluxo ETL (Extração, Transformação e Carregamento). O projeto foi adaptado para o universo de Naruto como uma alternativa criativa ao desafio original da Santander Dev Week.

Prepare as ferramentas e a sua chave da IA.

**Objetivo**
* Extrair informações detalhadas da organização criminosa Akatsuki, processar esses dados para gerar alertas de perigo personalizados e salvar os resultados em um arquivo CSV para consulta.

In [48]:
'!pip install -U google-genai'

'!pip install -U google-genai'

In [49]:
import requests
import pandas as pd
from google import genai
import time

# Configuração da IA
API_KEY = "Sua chave API_KEY"
client = genai.Client(api_key=API_KEY)
MODEL_ID = "gemini-1.5-flash"

1- **Extract** (Extração): Consumo da API dattebayo-api para obter dados brutos de membros da Akatsuki.
# Nova seção

In [50]:
def extract_akatsuki_data():
    print("Iniciando Extração: Acessando esconderijo da Akatsuki...")
    url = "https://dattebayo-api.onrender.com/akatsuki"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()['akatsuki']
    else:
        print("Erro ao acessar a API.")
        return []

# Executando a extração
raw_members = extract_akatsuki_data()
print(f"Sucesso! {len(raw_members)} ninjas extraídos.")

Iniciando Extração: Acessando esconderijo da Akatsuki...
Sucesso! 20 ninjas extraídos.


2. **Transform** (Transformação):  Tratamento de dados estruturados (Clã, Títulos e Afiliações).

* Uso de Inteligência Artificial (Google Gemini) para gerar
mensagens personalizadas do Mestre Kakashi.

* Implementação de lógica de fallback para garantir a entrega dos dados mesmo em caso de falha da API de IA.

In [52]:
def transform_ninja(ninja):
    # Coleta de dados
    personal = ninja.get('personal', {})
    nome = ninja.get('name', 'N/A')
    cla = personal.get('clan', 'N/A')
    classificacao = ", ".join(personal.get('classification', [])) if isinstance(personal.get('classification'), list) else "N/A"
    afiliacao = ", ".join(personal.get('affiliation', [])) if isinstance(personal.get('affiliation'), list) else "N/A"
    titulos = ", ".join(personal.get('titles', [])) if isinstance(personal.get('titles'), list) else "N/A"

    cla_info = cla if cla != "N/A" else "desconhecido"
    titulos_info = titulos if titulos != "N/A" else "não listados"

    prompt = (f"Aja como o Mestre Kakashi Hatake. Analise este ninja da Akatsuki:\n"
              f"- Nome: {nome}\n"
              f"- Clã: {cla_info}\n"
              f"- Títulos: {titulos_info}\n"
              f"Com base nisso, escreva um alerta rápido e sério para gennins sobre o perigo dele.")


    try:
        time.sleep(2)
        response = client.models.generate_content(
            model='gemini-1.5-flash',
            contents=prompt
        )
        mensagem = response.text.strip()
    except Exception as e:
        mensagem = f"CUIDADO! {nome} é um ninja perigoso com o título de {titulos}. Não se aproxime!"

    return {
        "Nome": nome,
        "Clã": cla,
        "Classificação": classificacao,
        "Afiliação": afiliacao,
        "Títulos": titulos,
        "Alerta_IA": mensagem
    }
# Aplicando a transformação nos 5 primeiros membros para teste rápido
print("Transformando dados e gerando alertas com IA...")
ninjas_processados = [transform_ninja(m) for m in raw_members[:10]]


Transformando dados e gerando alertas com IA...


3. Load (Carregamento)
Finalizamos salvando a tabela completa em um arquivo CSV.

In [53]:
def load_data(data):
    print("Iniciando Carregamento: Gerando Livro Bingo...")
    df = pd.DataFrame(data)

    # Salvando em CSV
    file_name = 'livro_bingo_akatsuki.csv'
    df.to_csv(file_name, index=False, encoding='utf-8')

    print(f"✅ Arquivo '{file_name}' gerado com sucesso!")
    return df

# Executando a carga final
df_final = load_data(ninjas_processados)

# Visualizando o resultado final
df_final.head()
print("\n--- RESULTADO FINAL ---")
print(df_final)

Iniciando Carregamento: Gerando Livro Bingo...
✅ Arquivo 'livro_bingo_akatsuki.csv' gerado com sucesso!

--- RESULTADO FINAL ---
                               Nome     Clã  \
0                       Black Zetsu     N/A   
1                             Chibi     N/A   
2                          Daibutsu     N/A   
3                           Deidara     N/A   
4  Demonic Statue of the Outer Path     N/A   
5                             Ginji     N/A   
6                             Hidan     N/A   
7                          Himeyuri     N/A   
8                            Hinoki     N/A   
9                     Itachi Uchiha  Uchiha   

                          Classificação               Afiliação  \
0       Jinchūriki, S-rank, Sensor Type                     N/A   
1                                   N/A                     N/A   
2                                   N/A                     N/A   
3  S-rank, Missing-nin, Mercenary Ninja     Iwagakure, Akatsuki   
4                 